# Random selection of tweets

This notebook contains the script and functions to generate random tweets selection according to the number of reviewers for each set and generate the output files in the most convinient format, in this case:
- 5000 tweets per set
- 5 reviewers per group

In [63]:
#####################
# Load Libraries    #
#####################
#install.packages("dplyr")
library("dplyr")

In [1]:
###################################
# Random tweets polarity selection #
###################################
#1st. Look for all the files that are in the folder 
#those tweets are clean tweets that we are using as input for the AWS comprehend polarity analysis
rm(list=ls())
fileList <- list.files("/home/ec2-user/SageMaker/CleanAndAggregateTweets/")

In [76]:
#2nd.we read all the files and we create a unique file with all of them together
for( i in 1:length( fileList ) ){
    if(i == 1){
        mytweetlist <- read.delim(paste0("/home/ec2-user/SageMaker/CleanAndAggregateTweets/", 
                                        fileList[i]), header = TRUE)
    }else{
        newtweetfile <- read.delim(paste0("/home/ec2-user/SageMaker/CleanAndAggregateTweets/", 
                                        fileList[i]), header = TRUE)
        mytweetlist <- rbind( mytweetlist, newtweetfile )
        rm( newtweetfile)
    }
}
print(paste0("In total we have ", nrow(mytweetlist), " tweets for the polarity analysis."))

[1] "In total we have 679443 tweets for the polarity analysis."


In [77]:
########################################
# Function to extract tweets randomly #
#######################################
randomTweetsPolarityExtraction <- function( tweets.df, tweetNumber, reviewersName, tweetSet, outputPath ){
    
    randomSelection <- sample_n( tweets.df, tweetNumber+1)
    
    write.table(randomSelection, file= paste0(outputPath, "polarity_set", tweetSet, "WithResults.csv"), col.names = TRUE, row.names = FALSE, sep="," )
    
    randomSelectionForReviewers <- as.data.frame(as.character(randomSelection$text))
    randomSelectionForReviewers$positive <- "-"
    randomSelectionForReviewers$negative <- "-"
    randomSelectionForReviewers$neutral <- "-"
    randomSelectionForReviewers$mixed <- "-"
    randomSelectionForReviewers$falsePositive <- "-"
    
    colnames(randomSelectionForReviewers)[1] <- "text"
    
    for( i in 1:length(reviewersName)){
          write.table(randomSelectionForReviewers, file= paste0(outputPath, reviewersName[i], "_polarity_set", tweetSet,".csv"), col.names = TRUE, row.names = FALSE, sep="," )
    }
}

We have 10 reviewers, so we will create two sets of 500 tweets each. 

In [78]:
randomTweetsPolarityExtraction( tweets.df   = mytweetlist,
                        tweetNumber = 500,
                        reviewersName = c("tomi", "annie", "sonya", "kim", "sofia"),
                        tweetSet = 1,
                        outputPath  = "/home/ec2-user/SageMaker/tweetsToReviewPolarity/" )

randomTweetsPolarityExtraction( tweets.df   = mytweetlist,
                        tweetNumber = 500,
                        reviewersName = c("melody", "anabel", "natalie", "sun", "kathleen", "chloe"),
                        tweetSet = 2,
                        outputPath  = "/home/ec2-user/SageMaker/tweetsToReviewPolarity/" )

In [11]:
sessionInfo()

R version 3.6.1 (2019-07-05)
Platform: x86_64-conda_cos6-linux-gnu (64-bit)
Running under: Amazon Linux AMI 2018.03

Matrix products: default
BLAS/LAPACK: /home/ec2-user/anaconda3/envs/R/lib/R/lib/libRblas.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] dplyr_0.8.0.1

loaded via a namespace (and not attached):
 [1] Rcpp_1.0.1       digest_0.6.18    crayon_1.3.4     assertthat_0.2.1
 [5] IRdisplay_0.7.0  repr_0.19.2      R6_2.4.0         jsonlite_1.6    
 [9] magrittr_1.5     evaluate_0.13    pillar_1.3.1     rlang_0.3.4     
[13] uuid_0.1-2       